# Introduction to Q# #


Q# (q-sharp) is a domain-specific and [open-source](https://github.com/microsoft/qsharp-language) programming language, part of [Microsoft's Quantum Development Kit (QDK)](https://docs.microsoft.com/en-us/quantum/), used for expressing quantum algorithms. It is to be used for writing integrated hybrid programs that leverage both classical and quantum processing on a quantum processing unit (QPU).




Operations and functions are the basic unit of execution in Q#. They are roughly equivalent to a function in C or C++ or Python, or a static method in C# or Java.

A Q# operation is a quantum subroutine. That is, it is a callable routine that contains quantum operations.

A Q# function is equivalent to a mathematical function, that is a complete deterministic subroutine used within a quantum algorithm that when invoked with the same arguments will always produce the same result. It may contain classical code but no quantum operations. Specifically, functions may not allocate or borrow qubits, nor may they call operations. It is possible, however, to pass them operations or qubits for processing.

Together, operations and functions are called callables.




IQ#, the Q# Jupyter kernel, allows you to write Q# functions and operations directly on a code cell and compile them by running the cell (pressing Ctrl/⌘+Enter). For example:

In [1]:
function Hello(name: String) : Unit {

    // The following line will simply write a message to the console:
    Message($"Hello {name}!");

}

Hello

When you **Run** the cell, Q# compiles the code and returns the name of the callables it found. 
In this case, it finds only one callable (`Hello`) that accepts one argument (`name` of type `String`) 
and returns no value (`Unit`, roughly equivalent to `void` in C).

As you can see, Q# is structurally very similar to familiar
languages such as C# and Java in its use of semicolons to end
statements, curly brackets to group statements, function calls and doubleslash to introduce comments. 
It is a strongly typed language so all variables, arguments and return values
must have an associated type. Q# supports a familar set of classical types like:
- Int
- BigInt
- Bool
- Double

and also some quantum-specific types:
- Qubit
- Result
- Pauli

The complete set of primitive types can be found in [Q# type model documentation](https://docs.microsoft.com/en-us/quantum/language/type-model?view=qsharp-preview#primitive-types).

In classical programs a bit, or binary digit, represents the basic unit of information and can be only one of two states: `0` or `1`.

Similarly, the **qubit**, or quantum bit, represents the basic unit of information for quantum algorithms. As we explained before, a qubit when measured can only be in one of two states however during execution, the state of the qubit represents *the probability of reading* $|0\rangle$ or $|1\rangle$. 

In Q#, $|0\rangle$ and $|1\rangle$ are represented by `Zero` and `One` accordingly and qubits can only be allocated inside an operation with the `use` statement. When it gets allocated a qubit is always in the $|0\rangle$  state. To measure a qubit and read its value you use the [`M`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.primitive.m) intrinsic operation.

As such, the next `HelloQuantum` operation always return `Zero`:

In [2]:
operation HelloQuantum() : Result {
    use q = Qubit();
    return M(q);
}

HelloQuantum

To verify this, you can use the `%simulate` magic to simulate the execution of the operation in a QPU:

In [3]:
%simulate HelloQuantum

Zero

A little more interesting example for a Q# operation is a quantum random bit generator (QRBG). A QRBG returns a different value each time it is invoked:

In [4]:
operation QRBG() : Result {
    use q = Qubit();

    // Create superposition:
    H(q);

    // Assign the value of measuring the qubit to a variable
    let r = M(q);
    return r;
}

QRBG

`QRBG` demonstrates some basic Q# features:

- The `use` keyword allocates new qubits. The lifetime of the qubit is based on its scope, when the variable runs out of scope it is automatically deallocated.

- Q# includes all common basic quantum gates, like `X` and `H`.

- Immutable variables are created using `let`, in the previous example, the variable `r` is assigned the value returned by invoking `M` on qubit `q`.



To verify the behavior of `QRBG`, we can create another Q# operation that invokes it 10 times, using a `for` loop on the variable `i` on a `Range` from 1 to 10 (inclusive), and prints the result to the console:

In [5]:
operation Main() : Unit {
    for i in 1..10 {
        Message($"{i}: {QRBG()}");
    }
}

Main

In [6]:
%simulate Main

1: Zero
2: Zero
3: One
4: Zero
5: Zero
6: One
7: One
8: One
9: One
10: One


()

Q# operations can return more than a single `Result`, let's modify `QRBG` to return an array:

In [7]:
open Microsoft.Quantum.Arrays;

operation QRBG2(n: Int) : Result[] {    
    use qs = Qubit[n];

    mutable r = [Zero, size=n];
    
    ApplyToEach(H, qs);
        
    for i in IndexRange(qs) {
        set r w/= i <- M(qs[i]);
    }

    return r;
}

QRBG2

In [8]:
%simulate QRBG2 n=8

One, Zero, One, Zero, Zero, One, Zero, Zero

`QRBG2` demonstrates other Q# features:

- `mutable` is used to initialize variables that can be modified later in the code using `set`.
- `[Zero, size=n]` is used to initialize an array. The first parameter specifies the default value of all entries in the array, and the `size` parameter specifies the number of elements in the array. In this case we create an array of `Result` elements with value `Zero` of size `n`.
- As other languages, `[]` is used to access the element of an array.
- New arrays can be created from existing ones via [copy-and-update expressions](https://docs.microsoft.com/en-us/quantum/language/expressions?view=qsharp-preview#copy-and-update-expressions). A copy-and-update expression is an expression of the form `arr w/ idx <- value` that constructs a new array with all elements set to the corresponding element in `arr`, except for the element(s) at `idx`, which are set to the one(s) in `value`. The resulting array can be assigned to the same variable by using the `w/=` operator.


- Q# has a rich set of built-in libraries, for this example we're using:

  -  [`ApplyToEach`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.canon.applytoeach): is an Operation that receives another operation as paramter and an array, and applies the given operation to each element of the array. In this particular case
  -  [`IndexRange`](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.arrays.indexrange): is a function that given an array, creates a range to iterate over the indices of the elements in the array.


- All callables belong to a namespace. In Jupyter, you can use all the operations in the [Microsoft.Quantum.Intrinsic](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.intrinsic?view=qsharp-preview) and the [Microsoft.Quantum.Canon](https://docs.microsoft.com/en-us/qsharp/api/qsharp/microsoft.quantum.canon?view=qsharp-preview) namespaces automatically. To use operations in other namespaces, like `IndexRange` from `Microsoft.Quantum.Array`, you have to use the fully qualified name of the operation (i.e. `Microsoft.Quantum.Array.IndexRange`) or include an `open` statement at the top.


### Oracles

Oracles are common in quantum algorithms. An Oracle is a function that returns `True` only when the input matches a criteria.

In [9]:
// Returns True when expected == actual
function AreEqual(expected: Int, actual: Int) : Bool {
    return expected == actual;
}

// A simple operation that accepts an oracle as parameter and applies `X`
// only when the oracle returns True for the given input value.
operation FlipOnOracle(q: Qubit, v: Int, oracle: Int -> Bool) : Unit {
    if oracle(v) {
        X(q);
    }
}

// Creates and returns an oracle by using partial application:
function CreateOracle() : Int -> Bool {
    return AreEqual(6, _);
}

operation FlipOnValue(value: Int) : Result {
    use q = Qubit();
    // Create a new function 
    let oracle = CreateOracle();
    FlipOnOracle(q, value, oracle);
    return M(q);
}

AreEqual, FlipOnOracle, CreateOracle, FlipOnValue

In [10]:
%simulate FlipOnValue value=2

Zero

In [11]:
%simulate FlipOnValue value=6

One

Callables are first class object types in Q#.

- Functions Types are specified as `(input_type) -> (output_type)`
- Operation Types are specified as `(input_type) => (output_type)`
- They can be used as any other data type.
  * as input parameters: `oracle` in `FlipOnOracle` is an input parameter.
  * as return values: `CreateOracle` returns a new function creaed using *partial application*.
  * as variables: the function returned by `CreateOracle` is assigned to `oracle` inside the `FlipOnValue` function
- *Partial application* allows to create new functions and operations by calling the function leaving certain parameters undetermined by passing `_`. For example `AreEqual(6, _)` returns a new function with only one input (i.e. `Int -> Bool`) that is equivalent to calling `AreEqual` with the first parameter always set to 6.



### Running on the cloud

So far, we've been simulating our Q# operations using the Jupyter kernel's built-in simulator. Connecting to an Azure Quantum Workspace allows you to run your Q# operations on a cloud simulator, or even on an actual QPU.

In [12]:
%azure.connect 
"/subscriptions/916dfd6d-030c-4bd9-b579-7bb6d1926e97/resourceGroups/anpaz-demos/providers/Microsoft.Quantum/Workspaces/demo-sim"
location=westus2

Authenticated using Azure.Identity.AzureCliCredential


Connected to Azure Quantum workspace demo-sim in location westus2.


Target ID,Current Availability,Average Queue Time (Seconds)
ionq.qpu,Available,194
ionq.simulator,Available,2


In [ ]:
%azure.target ionq.simulator

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.simulator


Target ID,Current Availability,Average Queue Time (Seconds)
ionq.simulator,Available,2


First you connect to your workspace by using the `%azure.connect` magic which expects two parameters that you can obtain from the Azure Portal
1. A resource-id
2. A location

If connected successfully, you get the list of targets available to execute your Q# operations. Using the `%azure.target` magic you select one to submit jobs. In this case, we selected the `ionq.simulator` to validate the operation before running to the QPU.

In [25]:
%azure.submit QRBG2 n=3

Submitting QRBG2 to target ionq.simulator...
Job successfully submitted for 500 shots.
   Job name: QRBG2
   Job ID: e10d344b-0838-4009-acee-1f0159a2945f


Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time
QRBG2,e10d344b-0838-4009-acee-1f0159a2945f,Waiting,ionq.simulator,3/2/2022 4:59:59 AM +00:00,,


`%azure.submit` works like `%simulate`, except that instead of simulating it locally sends the job for execution on the cloud. `%azure.submit` returns a job-id that you can use to check status using the `%azure.status` magic, or get the results once the job finishes execution by using `%azure.output`.

In [27]:
%azure.status

Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time
QRBG2,e10d344b-0838-4009-acee-1f0159a2945f,Succeeded,ionq.simulator,3/2/2022 4:59:59 AM +00:00,3/2/2022 5:00:02 AM +00:00,3/2/2022 5:00:02 AM +00:00


In [16]:
%azure.output

Result,Frequency,Histogram
"[0,0,0]",0.125,
"[1,0,0]",0.125,
"[0,1,0]",0.125,
"[1,1,0]",0.125,
"[0,0,1]",0.125,
"[1,0,1]",0.125,
"[0,1,1]",0.125,
"[1,1,1]",0.125,


Now, run against the quantum device:

In [20]:
%azure.target ionq.qpu

Loading package Microsoft.Quantum.Providers.IonQ and dependencies...
Active target is now ionq.qpu


Target ID,Current Availability,Average Queue Time (Seconds)
ionq.qpu,Available,194


In [21]:
%azure.submit QRBG2 n=3

Submitting QRBG2 to target ionq.qpu...
Job successfully submitted for 500 shots.
   Job name: QRBG2
   Job ID: b08c4b80-a11f-4d99-9d39-45969fce722f


Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time
QRBG2,b08c4b80-a11f-4d99-9d39-45969fce722f,Waiting,ionq.qpu,3/2/2022 4:54:01 AM +00:00,,


This might take a while. Query the state until the output changes to `Succeeded`

In [30]:
%azure.status b08c4b80-a11f-4d99-9d39-45969fce722f

Job Name,Job ID,Job Status,Target,Creation Time,Begin Execution Time,End Execution Time
QRBG2,b08c4b80-a11f-4d99-9d39-45969fce722f,Succeeded,ionq.qpu,3/2/2022 4:54:01 AM +00:00,3/2/2022 5:43:28 AM +00:00,3/2/2022 5:43:31 AM +00:00


In [31]:
%azure.output b08c4b80-a11f-4d99-9d39-45969fce722f

Result,Frequency,Histogram
"[0,0,0]",0.118,
"[1,0,0]",0.152,
"[0,1,0]",0.08,
"[1,1,0]",0.114,
"[0,0,1]",0.158,
"[1,0,1]",0.154,
"[0,1,1]",0.108,
"[1,1,1]",0.116,


### Getting Help ###

Q# supports adding documentation to operations via comments in the code. When such documentation exists, you can access it from the notebook by adding a question mark before or after the operation name on a code cell, for example:

In [17]:
Microsoft.Quantum.Intrinsic.X?

Microsoft.Quantum.Intrinsic.X

In [18]:
%who

AreEqual, CreateOracle, FlipOnOracle, FlipOnValue, Hello, HelloQuantum, Main, QRBG, QRBG2

`%who` returns the list of all local and workspace operations available.

In [19]:
%trace QRBG2 n=4

Microsoft.Quantum.IQSharp.Jupyter.DisplayableHtmlElement

`%trace` trace simulates an operation and returns a visualization of the quantum circuit that got executed.

# Next

[A quantum algorithm](./3.QuantumAlgorithm.ipynb)